# Sensitivity of the parameters.

In [1]:
import pandas as pd 

import numpy as np

import math

import matplotlib.pyplot as plt 

from pylab import *

from scipy import integrate

# Integral model.

In [2]:
from scipy.integrate import quad

def lamb(t,c2):
    c1, c3 = 220, 1650
    return 8*c1*math.exp(-(t-c2)**2/c3)/500000

def phi(t,c2):
    return quad(lamb,0,t, args=(c2))[0]

def S(t,S_0,nu,c2):
    return S_0*math.exp(-phi(t,c2) - nu*t)

def P_l(t,S_0,q,nu, c2):
    return S_0*(1-q)*(math.exp(-phi(t,c2))-math.exp(-phi(t, c2) - nu*t))

def g_p(t, S_0,q,nu, c2):
    return P_l(t,S_0,q,nu, c2)*lamb(t,c2)

def I_p(T,S_0,q,nu,c2):
    return quad(g_p,0,T,args=(S_0,q,nu,c2))[0]

def g(t,nu,sigma,c2):
    return -phi(t,c2)-(nu*t) + (sigma*t)

def g_v(t,nu,sigma,c2):
    return math.exp(g(t,nu,sigma,c2))

#def g_v2(t,T,nu,sigma):#    return math.exp(-phi(t)-(nu*t) + (sigma*(t-T)))

def V(T,S_0,q,nu,sigma,c2):
    return q*S_0*nu*math.exp(-sigma*T)*quad(g_v,0,T,args=(nu, sigma,c2))[0]

def g_f(t,S_0,q,nu,sigma,c2):
    return V(t,S_0,q,nu,sigma,c2)*math.exp(phi(t,c2)) 

def F(T,S_0,q,nu,sigma,w,c2):
    return (1-w)*sigma*math.exp(-phi(T,c2))*quad(g_f,0,T,args=(S_0,q,nu,sigma,c2))[0]

def P(T,S_0,q,nu,sigma,w,c2):
    return w*sigma*(quad(V,0,T, args=(S_0,q,nu,sigma,c2))[0])

def g_iv(t,S_0,q,nu,sigma,w,c2):
    return lamb(t,c2)*F(t,S_0,q,nu,sigma,w,c2)

def I_v(T,S_0,q,nu,sigma,w,c2):
    return quad(g_iv,0,T,args=(S_0,q,nu,sigma,w,c2))[0]

In [3]:
S_0=20000
c2=100
q, w, nu, sigma = 0.5, 0.7, 0.028, 0.1
#P(100,S_0,q,nu,sigma,w)
def pba(t,nu,sigma):
    return t
T=2000
pba(1000,nu,sigma)
quad(V,0,2000,args=(S_0,q,nu, sigma,c2))[0]

98022.50374382392

# T>0

# Parameters:
#    1.q
 #   2.w
 #   3.$\nu$
 #   4.$\sigma$

In [4]:

#Sensitivity of parameter q:

def sensibilidad_1(T,S_0,q,nu,sigma,w,c2):
    dp_l=-P_l(T,S_0,0,nu,c2)
    #dv=V(T,S_0,1,nu, sigma)
    df=F(T,S_0,1,nu,sigma,w,c2)
    dp=P(T,S_0,1,nu,sigma,w,c2)
    di_v=I_v(T,S_0,1,nu,sigma,w,c2)
    di_p=-I_p(T,S_0,0,nu,c2)
    p_l=P_l(T,S_0,q,nu,c2)
    v=V(T,S_0,q,nu, sigma,c2)
    f=F(T,S_0,q,nu,sigma,w,c2)
    p=P(T,S_0,q,nu,sigma,w,c2)
    i_v=I_v(T,S_0,q,nu,sigma,w,c2)
    i_p=I_p(T,S_0,q,nu,c2)
    
    A=((i_v*p_l)*(di_p*(i_v+f)+i_p*dp))/(i_p*(i_v+f+p))
    B=((i_p+p_l)*(i_v*(dp+df)-di_v*(p+f)))/(i_v+f+p)
    C=i_v*dp_l
    D=q/(i_p*(p+f)-(i_v*p_l))
    
    return (A+B-C)*D

In [5]:
#Sensitivity of parameter w:

def sensibilidad_2(T,S_0,q,nu,sigma,w,c2):
    dp_l=0
    df=-F(T,S_0,q,nu,sigma,0,c2)
    dp=P(T,S_0,q,nu,sigma,1,c2)
    di_v=-I_v(T,S_0,q,nu,sigma,0,c2)
    di_p=0
    p_l=P_l(T,S_0,q,nu,c2)
    f=F(T,S_0,q,nu,sigma,w,c2)
    p=P(T,S_0,q,nu,sigma,w,c2)
    i_v=I_v(T,S_0,q,nu,sigma,w,c2)
    i_p=I_p(T,S_0,q,nu,c2)
    
    A=((i_v*p_l)*(di_p*(i_v+f)+i_p*dp))/(i_p*(i_v+f+p))
    B=((i_p+p_l)*(i_v*(dp+df)-di_v*(p+f)))/(i_v+f+p)
    D=w/(i_p*(p+f)-(i_v*p_l))
    
    return (A+B)*D

In [6]:
#Sensitivity of parameter nu:

def g_dv(t,nu,sigma,c2):
    return t*g_v(t,nu,sigma,c2)

def dV_nu(T,S_0,q,nu,sigma,c2):
    A=-q*S_0*nu*math.exp(-sigma*T)*quad(g_dv,0,T,args=(nu, sigma,c2))[0]
    return A+(V(T,S_0,q,nu,sigma,c2)/nu)

def g_df(t,S_0,q,nu,sigma,c2):
    return dV_nu(t,S_0,q,nu,sigma,c2)*math.exp(phi(t,c2)) 

def dF_nu(T,S_0,q,nu,sigma,w,c2):
    return (1-w)*sigma*math.exp(-phi(T,c2))*quad(g_df,0,T,args=(S_0,q,nu,sigma,c2))[0]

def g_div(t,S_0,q,nu,sigma,w,c2):
    return lamb(t,c2)*dF_nu(t,S_0,q,nu,sigma,w,c2)

def g_dip(t,S_0,q,nu,sigma,c2):
    return t*(1-q)*S(t,S_0,nu,c2)*lamb(t,c2)

def dIp_nu(T,S_0,q,nu,sigma,c2):
    return quad(g_dip,0,T,args=(S_0,q,nu,sigma,c2))[0]

def sensibilidad_3(T,S_0,q,nu,sigma,w,c2):
    dp_l=T*(1-q)*S(T,S_0,nu,c2)
    df=dF_nu(T,S_0,q,nu,sigma,w,c2)
    dp=sigma*w*quad(dV_nu,0,T,args=(S_0,q,nu,sigma,c2))[0]
    di_v=quad(g_div,0,T,args=(S_0,q,nu,sigma,w,c2))[0]
    di_p=dIp_nu(T,S_0,q,nu,sigma,c2)
    p_l=P_l(T,S_0,q,nu,c2)
    f=F(T,S_0,q,nu,sigma,w,c2)
    p=P(T,S_0,q,nu,sigma,w,c2)
    i_v=I_v(T,S_0,q,nu,sigma,w,c2)
    i_p=I_p(T,S_0,q,nu,c2)
    
    A=((i_v*p_l)*(di_p*(i_v+f)+i_p*dp))/(i_p*(i_v+f+p))
    B=((i_p+p_l)*(i_v*(dp+df)-di_v*(p+f)))/(i_v+f+p)
    C=i_v*dp_l
    D=nu/(i_p*(p+f)-(i_v*p_l))
    
    return (A+B-C)*D

In [7]:
#Sensitivity of parameter sigma:

def dV_sig(T,S_0,q,nu,sigma,c2):
    A=q*S_0*nu*math.exp(-sigma*T)*quad(g_dv,0,T,args=(nu, sigma,c2))[0]
    return A-(sigma*V(T,S_0,q,nu,sigma,c2))

def f_df(t,S_0,q,nu,sigma,c2):
    return dV_sig(t,S_0,q,nu,sigma,c2)*math.exp(phi(t,c2)) 

def dF_sig(T,S_0,q,nu,sigma,w,c2):
    A=(1-w)*sigma*math.exp(-phi(T,c2))*quad(f_df,0,T,args=(S_0,q,nu,sigma,c2))[0]
    return A+ (F(T,S_0,q,nu,sigma,w,c2)/sigma)

def f_div(t,S_0,q,nu,sigma,w,c2):
    return lamb(t,c2)*dF_sig(t,S_0,q,nu,sigma,w,c2)

def sensibilidad_4(T,S_0,q,nu,sigma,w,c2):
    dp_l=0
    df=dF_sig(T,S_0,q,nu,sigma,w,c2)
    dp=w*(quad(V,0,T, args=(S_0,q,nu,sigma,c2))[0]) + sigma*w*quad(dV_sig,0,T,args=(S_0,q,nu,sigma,c2))[0]
    di_v=quad(f_div,0,T,args=(S_0,q,nu,sigma,w,c2))[0]
    di_p=0
    p_l=P_l(T,S_0,q,nu,c2)
    v=V(T,S_0,q,nu, sigma,c2)
    f=F(T,S_0,q,nu,sigma,w,c2)
    p=P(T,S_0,q,nu,sigma,w,c2)
    i_v=I_v(T,S_0,q,nu,sigma,w,c2)
    i_p=I_p(T,S_0,q,nu,c2)
    
    A=((i_v*p_l)*(di_p*(i_v+f)+i_p*dp))/(i_p*(i_v+f+p))
    B=((i_p+p_l)*(i_v*(dp+df)-di_v*(p+f)))/(i_v+f+p)
    C=i_v*dp_l
    D=sigma/(i_p*(p+f)-(i_v*p_l))
    
    return (A+B-C)*D

In [8]:
#Sensitivity of parameter lambda:

def dV_lamb(T,S_0,q,nu,sigma,c2):
    return -q*S_0*nu*math.exp(-sigma*T)*quad(g_dv,0,T,args=(nu, sigma,c2))[0]

def dF_lamb(t,S_0,q,nu,sigma,w,c2):
    return math.exp(-phi(t,c2))*(t*V(t,S_0,q,nu,sigma,c2)+dV_lamb(t,S_0,q,nu,sigma,c2))

def DF_lam(T,S_0,q,nu,sigma,w,c2): #Derivada de F.
    return -T*F(T,S_0,q,nu,sigma,w,c2)+(1-w)*sigma*math.exp(-phi(T,c2))*quad(dF_lamb,0,T,args=(S_0,q,nu,sigma,w,c2))[0]

def DF_lam_2(T,S_0,q,nu,sigma,w,c2):
    return lamb(T,c2)*DF_lam(T,S_0,q,nu,sigma,w,c2)

def dI_p(t,S_0,q,nu, c2):
    return P_l(t,S_0,q,nu,c2) + t*lamb(t,c2)*P_l(t,S_0,q,nu, c2) 

def sensibilidad_5(T,S_0,q,nu,sigma,w,c2):
    dp_l=-T*P_l(T,S_0,q,nu,c2)
    df=DF_lam(T,S_0,q,nu,sigma,w,c2)
    dp=sigma*w*quad(dV_lamb,0,T,args=(S_0,q,nu,sigma,c2))[0]
    di_v=quad(F,0,T,args=(S_0,q,nu,sigma,w,c2))[0] + quad(DF_lam_2,0,T,args=(S_0,q,nu,sigma,w,c2))[0]
    di_p=quad(dI_p,0,T,args=(S_0,q,nu,c2))[0]
    p_l=P_l(T,S_0,q,nu,c2)
    f=F(T,S_0,q,nu,sigma,w,c2)
    p=P(T,S_0,q,nu,sigma,w,c2)
    i_v=I_v(T,S_0,q,nu,sigma,w,c2)
    i_p=I_p(T,S_0,q,nu,c2)
    
    A=((i_v*p_l)*(di_p*(i_v+f)+i_p*dp))/(i_p*(i_v+f+p))
    B=((i_p+p_l)*(i_v*(dp+df)-di_v*(p+f)))/(i_v+f+p)
    C=i_v*dp_l
    D=lamb(T,c2)/(i_p*(p+f)-(i_v*p_l))
    
    return (A+B-C)*D

In [9]:
#Sensitivity of parameter tau:

def D_lam(t,c2): #Derivative of the incidence function.
    return 2*(t-c2)*lamb(t,c2)/1650

def int_lam(T,c2): #Derivative of the integral of the incidence.
    return quad(D_lam,0,T,args=(c2))[0]

def DV_c2(t,nu,sigma,c2):
    return int_lam(t,c2)*math.exp(g(t,nu,sigma,c2))

def dV_c2(T,S_0,q,nu,sigma,c2):#Derivative of V with respect to nu.
    return-q*S_0*nu*math.exp(-sigma*T)*quad(DV_c2,0,T,args=(nu, sigma,c2))[0]

def DF_c2(t,S_0,q,nu,sigma,c2):
    return math.exp(phi(t,c2))*(dV_c2(t,S_0,q,nu,sigma,c2)+(V(t,S_0,q,nu,sigma,c2)*int_lam(t,c2))) 

def dF_c2(T,S_0,q,nu,sigma,w,c2): # Derivative of F with respect to nu.
    return -int_lam(T,c2)*F(T,S_0,q,nu,sigma,w,c2)+(1-w)*sigma*math.exp(-phi(T,c2))*quad(DF_c2,0,T,args=(S_0,q,nu,sigma,c2))[0]

def DIV_c2(t,S_0,q,nu,sigma,w,c2):#Important function for derivative of I_v with respect to nu.
    return (D_lam(t,c2)*F(t,S_0,q,nu,sigma,w,c2))+ (lamb(t,c2)*dF_c2(T,S_0,q,nu,sigma,w,c2))

def DIP_c2(t,S_0,q,nu,sigma,w,c2):
    return -lamb(t,c2)*int_lam(T,c2)*P_l(T,S_0,q,nu, c2)+ D_lam(t,c2)*P_l(T,S_0,q,nu, c2)

def sensibilidad_6(T,S_0,q,nu,sigma,w,c2):
    dp_l=-int_lam(T,c2)*P_l(T,S_0,q,nu, c2)
    df=dF_c2(T,S_0,q,nu,sigma,w,c2)
    dp=sigma*w*quad(dV_c2,0,T,args=(S_0,q,nu,sigma,c2))[0]
    di_v=quad(DIV_c2,0,T,args=(S_0,q,nu,sigma,w,c2))[0]
    di_p=quad(DIP_c2,0,T,args=(S_0,q,nu,sigma,w,c2))[0]
    p_l=P_l(T,S_0,q,nu,c2)
    f=F(T,S_0,q,nu,sigma,w,c2)
    p=P(T,S_0,q,nu,sigma,w,c2)
    i_v=I_v(T,S_0,q,nu,sigma,w,c2)
    i_p=I_p(T,S_0,q,nu,c2)
    
    A=((i_v*p_l)*(di_p*(i_v+f)+i_p*dp))/(i_p*(i_v+f+p))
    B=((i_p+p_l)*(i_v*(dp+df)-di_v*(p+f)))/(i_v+f+p)
    C=i_v*dp_l
    D=c2/(i_p*(p+f)-(i_v*p_l))
    
    return (A+B-C)*D

In [10]:
T=60
c2=100
S_0, q, w, nu, sigma = 20000, 0.5, 0.7, 0.028, 0.1
sensibilidad_6(T,S_0,q,nu,sigma,w,c2)

1.5000969766184211

# Sensitivity calculations

In [11]:
vector=[50, 100,150, 200, 300, 500, 1000]
Par_1=[]
for k in vector:
    Par_1.append(round(sensibilidad_1(k,S_0,q,nu,sigma,w,c2),4))
    
Par_1
        
    

[0.5348, 0.5092, 0.4596, 0.4545, 0.4541, 0.4541, 0.4541]

In [12]:
vector=[50, 100,150, 200, 300, 500, 1000]
Par_2=[]
for k in vector:
    Par_2.append(round(sensibilidad_2(k,S_0,q,nu,sigma,w,c2),4))
    
Par_2
     

[1.1666, 1.209, 1.1901, 1.1846, 1.183, 1.1829, 1.1829]

In [13]:
vector=[50, 100,150, 200, 300, 500, 1000]
Par_3=[]
for k in vector:
    Par_3.append(round(sensibilidad_3(k,S_0,q,nu,sigma,w,c2),4))
    
Par_3
     

[-0.0007, -0.0185, -0.0324, -0.0438, -0.0499, -0.0506, -0.0506]

In [14]:
vector=[50, 100,150, 200, 300, 500]
Par_4=[]
for k in vector:
    Par_4.append(round(sensibilidad_4(k,S_0,q,nu,sigma,w,c2),4))
    
Par_4

[0.8193, 1.1042, 1.163, 1.2401, 1.2831, 1.2875]

In [15]:
vector=[50, 100,150, 200, 300, 500]
Par_5=[]
for k in vector:
    Par_5.append(round(sensibilidad_5(k,S_0,q,nu,sigma,w,c2),4))
    
Par_5

[-0.5827, -0.3821, -0.0603, -0.001, -0.0, -0.0]

In [16]:
vector=[50, 100,150, 200, 300, 500]
Par_6=[]
for k in vector:
    Par_6.append(round(sensibilidad_6(k,S_0,q,nu,sigma,w,c2),4))
    
Par_6

[1.6261, 0.6178, 0.0167, -0.0537, -0.0544, -0.0544]

In [17]:
c2=0
vector=[50, 100,150, 200, 300, 500]
Par_7=[]
for k in vector:
    Par_7.append(round(sensibilidad_6(k,S_0,q,nu,sigma,w,c2),4))
    
Par_7

[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0]

# T= $\infty$

In [18]:
T=5000
c2=100
S_0, q, w, nu, sigma = 20000, 0.5, 0.7, 0.028, 0.1

print(round(sensibilidad_1(T,S_0,q,nu,sigma,w,c2),4))
print(round(sensibilidad_2(T,S_0,q,nu,sigma,w,c2),4))
print(round(sensibilidad_3(T,S_0,q,nu,sigma,w,c2),4))
print(round(sensibilidad_4(T,S_0,q,nu,sigma,w,c2),4))
print(round(sensibilidad_5(T,S_0,q,nu,sigma,w,c2),4))
print(round(sensibilidad_6(T,S_0,q,nu,sigma,w,c2),4))

0.4541
1.1829
-0.0506
1.2876
-0.0
-0.0159
